# Advent of Code 2023

## Day 1

### Part 1

In [3]:
with open('inputs/day1.txt') as f:
    s = f.read()[:-1]

def parse(n):
    return int(n[0] + n[-1])

sum([parse([c for c in n if c.isnumeric()]) for n in s.split('\n')])

54634

### Part 2

In [4]:
numeral_map = {'one':'1',
               'two':'2',
               'three':'3',
               'four':'4',
               'five':'5',
               'six':'6',
               'seven':'7',
               'eight':'8',
               'nine':'9'}
numerals = list(numeral_map.keys()) + list(numeral_map.values())

def parse_numeral(n):
    return numeral_map[n] if n in numeral_map else n


def extract_number(s):
    indices_left = {n:s.find(n) for n in numerals if s.find(n) >= 0}
    indices_right = {n:s.rfind(n) for n in numerals if s.find(n) >= 0}
    leftmost = parse_numeral(min(indices_left,key=indices_left.get))
    rightmost = parse_numeral(max(indices_right,key=indices_right.get))
    return int(leftmost + rightmost)


sum([extract_number(n) for n in s.split('\n')])

53855

## Day 2

### Part 1

In [1]:
with open('inputs/day2.txt') as f:
    s = f.read()[:-1]

def parse_round(round):
    return {cubes.split(' ')[1]: int(cubes.split(' ')[0]) for cubes in round.split(', ')}


def parse_game(game):
    return [parse_round(round) for round in game.split(': ')[1].split('; ')]


def max_color_value(game, color):
    return (color, max([round.get(color,0) for round in game]))


def max_values(game):
    return dict(map(lambda c: max_color_value(game,c), ['red', 'green', 'blue']))


def is_valid(game,reds,greens,blues):
    maxes = max_values(game)
    return maxes.get('red',0) <= reds and maxes.get('green',0) <= greens and maxes.get('blue',0) <= blues


games = [parse_game(game) for game in s.split('\n')]
sum([i+1 for i,game in enumerate(games) if is_valid(game,12,13,14)])

2593

### Part 2

In [3]:
from functools import reduce

def game_power(game):
    return reduce(lambda a,b: a*b, max_values(game).values())


sum(map(game_power, games))

54699

## Day 3

### Part 1

In [114]:
from functools import reduce

with open('inputs/day3.txt') as f:
    s = f.read()[:-1].split('\n')

symbols = [(x,y) for y in range(len(s)) for x in range(len(s[0])) if not (s[y][x] == '.' or s[y][x].isnumeric())]
numbers = [(x,y) for y in range(len(s)) for x in range(len(s[0])) if s[y][x].isnumeric()]
neighbours = [(-1,-1), (-1,0), (-1,1), (0,-1), (0,1), (1,-1), (1,0), (1,1)]

def explore(reachable, current, target):
    (x,y) = current
    neighborhood = [(x+xd,y+yd) for (xd,yd) in neighbours]
    for coord in neighborhood:
        if coord in target and coord not in reachable:
            reachable.add(coord)
            explore(reachable, coord, target)
    return reachable


reachable = reduce(lambda acc,sym: explore(acc, sym, numbers), symbols, set())
number_starts = [(x,y) for (x,y) in reachable if (x-1,y) not in reachable]

def read_number(current, acc, reachable):
    (x,y) = current
    if current in reachable:
        return read_number((x+1,y), acc*10+int(s[y][x]), reachable)
    return acc


sum(map(lambda x: read_number(x,0,reachable), number_starts))

521515

### Part 2

In [116]:
symbols = [(x,y) for y in range(len(s)) for x in range(len(s[0])) if s[y][x] == '*']
reachable = {sym: explore(set(), sym, numbers) for sym in symbols}
numbers = list(reduce(lambda acc, v: acc.union(v), reachable.values(), set()))
number_starts = [(x,y) for (x,y) in numbers if (x-1,y) not in numbers]
reachable_number_starts = {sym: [n for n in reachable[sym] if n in number_starts] for sym in reachable}
gears = [reduce(lambda a,b: a*b, map(lambda x: read_number(x,0,numbers), num_starts), 1) 
         for num_starts in reachable_number_starts.values() if len(num_starts) == 2]

sum(gears)

69527306

## Day 4

### Part 1

In [39]:
with open('inputs/day4.txt') as f:
    s = f.read()[:-1]

input = [line.replace('  ', ' ').split(': ')[1].split(' | ') for line in s.split('\n')]
cards = [(list(map(int,card[0].split(' '))), list(map(int,card[1].split(' ')))) for card in input]
winning = [(len([x for x in card[1] if x in card[0]])) for card in cards]
sum([2 ** (card-1) for card in winning if card > 0])

21485

### Part 2

In [40]:
card_count = {i: 1 for i in range(len(cards))}
for i in range(len(cards)):
    for j in range(1, winning[i]+1):
        card_count[i+j] += card_count[i]

sum(card_count.values())

11024379

## Day 5

### Part 1

In [2]:
from functools import reduce

with open('inputs/day5.txt') as f:
    s = f.read()[:-1]

seeds = [int(seed) for seed in s.split('\n\n')[0].split(' ')[1:]]
maps = [[tuple(int(n) for n in m.split(' ')) for m in sec.split('\n')[1:]] for sec in s.split('\n\n')[1:]]

def attempt_map(n, map):
    destination_start, source_start, range_length = map
    if n >= source_start and n < source_start + range_length:
        return (destination_start + n - source_start, True)
    else:
        return (n, False)

def map_level(n,mapping):
    match = False
    i = -1
    while not match and i < len(mapping) - 1:
        i += 1
        result, match = attempt_map(n, mapping[i])
    return (result, i)


def map_full(n, levels):
    return reduce(lambda acc,level: map_level(acc[0],level), levels, (n,None))


min([map_full(seed, maps)[0] for seed in seeds])

484023871

### Part 2

In [3]:
from math import ceil

seed_ranges = [(seeds[i], seeds[i+1]) for i in range(0, len(seeds), 2)]
lowest_start = min([map_full(seed_range[0], maps) for seed_range in seed_ranges])

def find_buckets(seed_range, levels, buckets):
    (start, length) = seed_range
    if length <= 1:
        return buckets
    (result_1, last_map_1) = map_full(start, levels)
    (result_2, last_map_2) = map_full(start+length-1, levels)
    if last_map_1 not in buckets or buckets[last_map_1] > result_1:
        buckets[last_map_1] = result_1
    if last_map_2 not in buckets or buckets[last_map_2] > result_2:
        buckets[last_map_2] = result_2
    if last_map_1 != last_map_2:
        h1 = find_buckets((start, length//2), levels, buckets)
        h2 = find_buckets((start+length//2, ceil(length/2)), levels, buckets)
        for b in h1:
            if b not in buckets or buckets[b] > h1[b]:
                buckets[b] = h1[b]
        for b in h2:
            if b not in buckets or buckets[b] > h2[b]:
                buckets[b] = h2[b]
    return buckets


min(reduce(lambda acc,sr: find_buckets(sr,maps,acc), seed_ranges, {}).values())

46294175

## Day 6

### Part 1

In [34]:
from functools import reduce

with open('inputs/day6.txt') as f:
    s = f.read()[:-1]

[time, distance] = [[int(i) for i in ' '.join(line.split()).split(': ')[1].split(' ')] for line in s.split('\n')]

def distance_after(max_time, press_time):
    return (max_time-press_time)*press_time


reduce(lambda acc,i: acc*len([press for press in range(time[i]+1) if distance_after(time[i], press) > distance[i]]), range(len(time)), 1)

1624896

### Part 2

In [35]:
[time, distance] = [int(''.join(line.split(':')[1].split())) for line in s.split('\n')]
len([press for press in range(time+1) if distance_after(time, press) > distance])

32583852

## Day 7

### Part 1

In [89]:
with open('inputs/day7.txt') as f:
    s = f.read()[:-1].split('\n')


def count_chars(hand):
    chars = {}
    for c in hand:
        chars[c] = chars.get(c,0)+1
    return chars


def is_five_of_a_kind(hand):
    return 5 in count_chars(hand).values()


def is_four_of_a_kind(hand):
    return 4 in count_chars(hand).values()


def is_full_house(hand):
    values = count_chars(hand).values()
    return 3 in values and 2 in values


def is_three_of_a_kind(hand):
    values = count_chars(hand).values()
    return 3 in values and 2 not in values


def is_two_pairs(hand):
    values = count_chars(hand).values()
    return len([i for i in values if i == 2]) == 2


def is_pair(hand):
    values = count_chars(hand).values()
    return len([i for i in values if i == 2]) == 1 and 3 not in values


def is_high(hand):
    values = count_chars(hand).values()
    return len([i for i in values if i == 1]) == 5


types = [is_five_of_a_kind, is_four_of_a_kind, is_full_house, is_three_of_a_kind, is_two_pairs, is_pair, is_high]


def transform_hand(hand):
    return hand.replace('K', 'B').replace('Q','C').replace('J','D').replace('T','E') \
    .replace('9', 'F').replace('8','G').replace('7','H').replace('6','I').replace('5', 'J') \
    .replace('4','K').replace('3','L').replace('2','M').replace('1','N')


def normalize_hand(hand, bid):
    hand_t = transform_hand(hand)
    for i in range(len(types)):
        if types[i](hand_t):
            return ' '.join([str(i), hand_t, str(bid)])

hands = [normalize_hand(line.split(' ')[0], line.split(' ')[1]) for line in s]
hands.sort(reverse=True)

sum([(i+1)*int(hands[i].split(' ')[-1]) for i in range(len(hands))])

249483956

### Part 2

In [90]:
def transform_hand(hand):
    return hand.replace('K', 'B').replace('Q','C').replace('J','O').replace('T','E') \
    .replace('9', 'F').replace('8','G').replace('7','H').replace('6','I').replace('5', 'J') \
    .replace('4','K').replace('3','L').replace('2','M').replace('1','N')


def jokers(hand):
    return hand.count('O')


def count_chars(hand):
    chars = {}
    for c in hand:
        if c != 'O':
            chars[c] = chars.get(c,0)+1
    return chars


def is_five_of_a_kind(hand):
    joker_count = jokers(hand)
    return (5-jokers(hand) in count_chars(hand).values()) or joker_count == 5


def is_four_of_a_kind(hand):
    return 4-jokers(hand) in count_chars(hand).values()


def is_full_house(hand):
    values = count_chars(hand).values()
    joker_count = jokers(hand)
    return (joker_count == 0 and 3 in values and 2 in values) or \
    (joker_count == 1 and (3 in values or len([i for i in values if i == 2]) == 2))


def is_three_of_a_kind(hand):
    values = count_chars(hand).values()
    joker_count = jokers(hand)
    return (joker_count == 0 and 3 in values and 2 not in values) or \
    (joker_count == 1 and len([i for i in values if i == 2]) == 1) or \
    (joker_count == 2 and len([i for i in values if i == 1]) == 3)


def is_two_pairs(hand):
    values = count_chars(hand).values()
    return (len([i for i in values if i == 2]) == 2)


def is_pair(hand):
    values = count_chars(hand).values()
    joker_count = jokers(hand)
    return (joker_count == 0 and len([i for i in values if i == 2]) == 1 and 3 not in values) or \
    (joker_count == 1 and len([i for i in values if i == 1]) == 4)


def is_high(hand):
    values = count_chars(hand).values()
    return len([i for i in values if i == 1]) == 5


def normalize_hand(hand, bid):
    hand_t = transform_hand(hand)
    for i in range(len(types)):
        if types[i](hand_t):
            return ' '.join([str(i), hand_t, str(bid)])


types = [is_five_of_a_kind, is_four_of_a_kind, is_full_house, is_three_of_a_kind, is_two_pairs, is_pair, is_high]
hands = [normalize_hand(line.split(' ')[0], line.split(' ')[1]) for line in s]
hands.sort(reverse=True)

sum([(i+1)*int(hands[i].split(' ')[-1]) for i in range(len(hands))])

252137472

## Day 8

### Part 1

In [4]:
with open('inputs/day8.txt') as f:
    s = f.read()[:-1]

instructions = [0 if inst == 'L' else 1 for inst in s.split('\n')[0]]
nodes = {line.split(' = ')[0]: tuple(line.split(' = ')[-1][1:-1].split(', ')) for line in s.split('\n\n')[1].split('\n')}

current = 'AAA'
steps = 0
while current != 'ZZZ':
    current = nodes[current][instructions[steps % len(instructions)]]
    steps += 1

steps

12737

### Part 2

In [11]:
from functools import reduce
from math import lcm

def find_loop(start, instrtuctions, nodes):
    i = 0
    current = start
    visited = []
    ends = set()
    while (current, i % len(instructions)) not in visited:
        visited.append((current, i % len(instructions)))
        current = nodes[current][instructions[i % len(instructions)]]
        i += 1
        if current[-1] == 'Z':
            ends.add(i)
    loop_start = visited.index((current, i % len(instructions)))
    loop_length = i - loop_start
    return (loop_length, ends)


current_nodes = [node for node in nodes if node[-1] == 'A']
loops = [find_loop(start, instructions, nodes)[0] for start in current_nodes]
reduce(lambda acc,x: lcm(acc,x), loops, 1) # No further steps are necessary due to the input being a special case

9064949303801

## Day 9

### Part 1

In [48]:
with open('inputs/day9.txt') as f:
    s = f.read()[:-1].split('\n')

sequences = [[int(i) for i in line.split(' ')] for line in s]

def next_seq(seq):
    return [seq[i+1]-seq[i] for i in range(len(seq)-1)]


def is_last(seq):
    seq_set = set(seq)
    return (len(seq_set) == 1 and 0 in seq_set) or len(seq_set) == 0


def predict_next(seq):
    acc = 0
    while not is_last(seq):
        acc += seq[-1]
        seq = next_seq(seq)
    return acc


sum([predict_next(seq) for seq in sequences])

1882395907

### Part 2

In [62]:
def all_subsequences(seq):
    seqs = [seq]
    while not is_last(seqs[-1]):
        seqs.append(next_seq(seqs[-1]))
    return seqs


def predict_previous(seq):
    all = all_subsequences(seq)
    prev = 0
    for i in range(1,len(all)+1):
        current = all[-i][0] - prev
        prev = current
    return current


sum([predict_previous(seq) for seq in sequences])

1005

 ## Day 10

### Part 1

In [21]:
with open('inputs/day10.txt') as f:
    s = f.read()[:-1]

grid = s.split('\n')
neighbours = {'|': {(0,-1), (0,1)},
              '-': {(-1,0), (1,0)},
              'L': {(0,-1), (1,0)},
              'J': {(0,-1), (-1,0)},
              '7': {(0,1), (-1,0)},
              'F': {(0,1), (1,0)},
              'S': {(-1,0), (1,0), (0,-1), (0,1)},
              '.': {}}

for y,line in enumerate(grid):
    if 'S' in line:
        y0 = y
        x0 = line.index('S')

distances = {}
queue = {(x0,y0,0)}


def connections(x,y):
    return neighbours[grid[y][x]]


def connects_to(x0,y0,xd,yd):
    return (xd,yd) in connections(x0,y0) and (-xd,-yd) in connections(x0+xd, y0+yd)


def look_around(x,y):
    return [(x+xd,y+yd) for (xd,yd) in connections(x,y) if connects_to(x,y,xd,yd)]


def visit_neighbours(x,y,d):
    if (x,y) in distances and d >= distances[(x,y)]:
        return
    distances[(x,y)] = d
    connecting = look_around(x,y)
    for (x1,y1) in connecting:
        if (x1,y1) not in distances or distances[(x1,y1)] > d+1:
            queue.add((x1,y1,d+1))


while len(queue) > 0:
    x,y,d = queue.pop()
    visit_neighbours(x,y,d)


(xf,yf) = max(distances,key=distances.get)
distances[(xf,yf)]

6640

### Part 2

In [22]:
grid = [' '.join(list(y)) for y in s.split('\n')]
for i in range(len(grid)-1,-1,-1):
    grid.insert(i+1,' '*len(grid[0]))

neighbours = {'|': {(0,-2), (0,2)},
              '-': {(-2,0), (2,0)},
              'L': {(0,-2), (2,0)},
              'J': {(0,-2), (-2,0)},
              '7': {(0,2), (-2,0)},
              'F': {(0,2), (2,0)},
              'S': {(-2,0), (2,0), (0,-2), (0,2)},
              '.': {}}

for y,line in enumerate(grid):
    if 'S' in line:
        y0 = y
        x0 = line.index('S')


def connects_to(x0,y0,xd,yd):
    connects = (xd,yd) in connections(x0,y0) and (-xd,-yd) in connections(x0+xd, y0+yd)
    if connects:
        grid[y0+yd//2] = grid[y0+yd//2][:x0+xd//2] + 'x' + grid[y0+yd//2][x0+xd//2+1:]
    return connects


distances = {}
queue = {(x0,y0,0)}
while len(queue) > 0:
    x,y,d = queue.pop()
    visit_neighbours(x,y,d)

loop = [''.join(['.' if ((x,y) not in distances and grid[y][x] != 'x' and x % 2 == 0 and y % 2 == 0) else grid[y][x] \
                 for x in range(len(grid[0]))]) for y in range(len(grid))]

def expand(x0,y0):
    queue = {(x0,y0)}
    visited = set()
    while len(queue) > 0:
        (x,y) = queue.pop()
        if x >= 0 and x < len(grid[0]) and y >= 0 and y < len(grid) and (loop[y][x] == '.' or loop[y][x] == ' ') and (x,y) not in visited:
            visited.add((x,y))
            queue.add((x-1,y))
            queue.add((x+1,y))
            queue.add((x,y-1))
            queue.add((x,y+1))
    return visited

outside = expand(0,0)
inside = [''.join([' ' if (x,y) in outside else loop[y][x] for x in range(len(loop[0]))]) for y in range(len(loop))]

sum([line.count('.') for line in inside])

411

## Day 11

### Part 1

In [78]:
with open('inputs/day11.txt') as f:
    s = f.read()[:-1]


def transpose(matrix):
    return [''.join([matrix[y][x] for y in range(len(matrix))]) for x in range(len(matrix[0]))]


space = s.split('\n')
space_t = transpose(space)
empty_y = {y for y in range(len(space)) if '#' not in space[y]}
empty_x = {x for x in range(len(space_t)) if '#' not in space_t[x]}
galaxies = [(x,y) for x in range(len(space[0])) for y in range(len(space)) if space[y][x] == '#']


def distance(g1,g2,empty_value):
    (x1,y1) = g1
    (x2,y2) = g2
    acc = abs(x1-x2) + abs(y1-y2)
    for x in empty_x:
        if x in range(min(x1,x2), max(x1,x2)):
            acc += empty_value-1
    for y in empty_y:
        if y in range(min(y1,y2), max(y1,y2)):
            acc += empty_value-1
    return acc


acc = 0
for i in range(len(galaxies)):
    for j in range(i+1, len(galaxies)):
        acc += distance(galaxies[i], galaxies[j],2)

acc

9370588

### Part 2

In [79]:
acc = 0
for i in range(len(galaxies)):
    for j in range(i+1, len(galaxies)):
        acc += distance(galaxies[i], galaxies[j],1000000)

acc

746207878188

## Day 12

### Part 1

In [298]:
from functools import cache

with open('inputs/day12.txt') as f:
    s = f.read()[:-1]

@cache
def count_solutions(record, broken_blocks, solutions):
    if len(record) == 0 and len(broken_blocks) == 0:
        return 1
    if len(record) == 0 and len(broken_blocks) > 0:
        return 0
    if record[0] == '.':
        return count_solutions(record[1:], broken_blocks, solutions)
    if record[0] == '#':
        if len(broken_blocks) == 0 or len(record) < broken_blocks[0] or '.' in record[:broken_blocks[0]] or \
           (len(record) > broken_blocks[0] and record[broken_blocks[0]] == '#'):
            return 0
        else:
            return count_solutions('.' + record[broken_blocks[0]+1:], broken_blocks[1:], solutions)
    return count_solutions('#' + record[1:], broken_blocks, solutions+1) + count_solutions('.' + record[1:], broken_blocks, solutions+1)


springs = [(line.split(' ')[0], tuple([int(n) for n in line.split(' ')[1].split(',')])) for line in s.split('\n')]
sum([count_solutions(spring[0],spring[1],0) for spring in springs])

7204

### Part 2

In [299]:
def unfold(spring):
    (record, broken_blocks) = spring
    return ('?'.join([record]*5),broken_blocks*5)

unfolded_springs = [unfold(spring) for spring in springs]
sum([count_solutions(spring[0],spring[1],0) for spring in unfolded_springs])

1672318386674

## Day 13

### Part 1

In [89]:
with open('inputs/day13.txt') as f:
    s = f.read()[:-1]

def transpose(matrix):
    return [''.join([matrix[y][x] for y in range(len(matrix))]) for x in range(len(matrix[0]))]


def check_reflection(pattern,y):
    size = min(y, len(pattern)-y)
    return pattern[y-size:y] == pattern[y:y+size][::-1] 


def find_horizontal_reflection(pattern):
    for y in range(1,len(pattern)):
        if check_reflection(pattern,y):
            return y
    return 0


def find_vertical_reflection(pattern):
    pattern_t = transpose(pattern)
    for x in range(1,len(pattern_t)):
        if check_reflection(pattern_t,x):
            return x
    return 0
    

def find_reflection_index(pattern):
    return (find_vertical_reflection(pattern), find_horizontal_reflection(pattern))


def reflection_value(pattern):
    (ref_x, ref_y) = find_reflection_index(pattern)
    return 100*ref_y + ref_x


patterns = [pattern.split('\n') for pattern in s.split('\n\n')]
sum(map(lambda pattern: reflection_value(pattern), patterns))

26957

### Part 2

In [124]:
def flip(pattern,x,y):
    replacement = '.' if pattern[y][x] == '#' else '#'
    return [pattern[yy][:x] + replacement + pattern[yy][x+1:] if yy == y else pattern[yy] for yy in range(len(pattern))]

def find_horizontal_reflection_except(pattern,ex):
    for y in range(1,len(pattern)):
        if check_reflection(pattern,y) and y != ex:
            return y
    return 0


def find_vertical_reflection_except(pattern,ex):
    pattern_t = transpose(pattern)
    for x in range(1,len(pattern_t)):
        if check_reflection(pattern_t,x) and x != ex:
            return x
    return 0
    

def find_reflection_index_except(pattern,ex_x,ex_y):
    return (find_vertical_reflection_except(pattern,ex_x), find_horizontal_reflection_except(pattern,ex_y))


def reflection_value_smudged(pattern):
    (ref_x_0, ref_y_0) = find_reflection_index(pattern)
    for y in range(len(pattern)):
        for x in range(len(pattern[0])):
            cleaned = flip(pattern,x,y)
            (ref_x, ref_y) = find_reflection_index_except(cleaned, ref_x_0, ref_y_0)
            if ref_x == ref_x_0:
                ref_x = 0
            if ref_y == ref_y_0:
                ref_y = 0
            if ref_x + ref_y != 0:
                return 100*ref_y + ref_x


sum(map(lambda pattern: reflection_value_smudged(pattern), patterns))

42695

## Day 14

### Part 1

In [94]:
with open('inputs/day14.txt') as f:
    s = f.read()[:-1].split('\n')

min_x = 0
min_y = 0
max_x = len(s[0])-1
max_y = len(s)-1

static = {(x,y) for y in range(max_y+1) for x in range(max_x+1) if s[y][x] == '#'}
rolling = {(x,y) for y in range(max_y+1) for x in range(max_x+1) if s[y][x] == 'O'}


def discover_stuck(rolling, direction):
    (x_d,y_d) = direction
    stuck = {(x,y) for (x,y) in rolling if (x+x_d) < min_x or (y+y_d) < min_y or (x+x_d) > max_x or (y+y_d) > max_y or (x+x_d,y+y_d) in static}
    added = len(stuck)
    while added != 0:
        also_stuck = {(x,y) for (x,y) in rolling if (x+x_d,y+y_d) in stuck and (x,y) not in stuck}
        added = len(also_stuck)
        stuck.update(also_stuck)
    return stuck


def roll_once(rolling, direction):
    (x_d,y_d) = direction
    stuck = discover_stuck(rolling, direction)
    free = {(x+x_d,y+y_d) for (x,y) in rolling if (x,y) not in stuck}
    new_rolling = stuck.union(free)
    return (new_rolling, rolling != new_rolling)


def full_roll(rolling, direction):
    (rolling_new, changed) = roll(rolling, direction)
    while changed:
        (rolling_new, changed) = roll(rolling_new, direction)
    return rolling_new


def load(rolling):
    return sum([max_y+1-y for (_,y) in rolling])

rolling_new = full_roll(rolling, (0,-1))
load(rolling_new)

108144

### Part 2

In [95]:
cache = {}

def to_tuple(rolling):
    return tuple(sorted(list(rolling)))


def cycle(rolling,i):
    rolling_t = to_tuple(rolling)
    if rolling_t in cache:
        return cache[rolling_t]
    rolling = full_roll(rolling, (0,-1))
    rolling = full_roll(rolling, (-1,0))
    rolling = full_roll(rolling, (0,1))
    rolling = full_roll(rolling, (1,0))
    cache[rolling_t] = (rolling,i)
    return (rolling,None)

(rolling_new,cache_hit) = cycle(rolling, 0)
cycles = 1000000000
i = 1
while i < cycles:
    (rolling_new, first_seen) = cycle(rolling_new,i)
    if first_seen is not None:
        loop_length = i-first_seen
        i = first_seen + loop_length*((cycles-first_seen)//loop_length)
    i += 1

load(rolling_new)

108404

## Day 15

### Part 1

In [32]:
from functools import reduce

with open('inputs/day15.txt') as f:
    s = f.read()[:-1].split(',')


def process_char(current, char):
    return ((current + ord(char))*17)%256


def hash(str):
    return reduce(lambda current,char: process_char(current,char), str, 0)


sum(map(lambda instruction: hash(instruction), s))

515210

### Part 2

In [33]:
def get_label(str):
    return str[:-1] if str[-1] == '-' else str[:-2]


def execute(boxes, instruction):
    label = get_label(instruction)
    box_index = hash(label)
    if instruction[-1] == '-':
        boxes[box_index] = [lens for lens in boxes[box_index] if lens[0] != label]
    else:
        focal_length = int(instruction[-1])
        if label not in map(lambda lens: lens[0], boxes[box_index]):
            boxes[box_index] += [(label, focal_length)]
        else:
            boxes[box_index] = [lens if lens[0] != label  else (label,focal_length) for lens in boxes[box_index]]
    return boxes


def box_power(boxes,index):
    box = boxes[index]
    return sum(map(lambda lens: (index+1)*lens[0]*lens[1][1], enumerate(box,1)))


boxes = reduce(lambda boxes, instruction: execute(boxes,instruction), s, [[] for _ in range(256)])
sum(map(lambda box_index: box_power(boxes, box_index), range(256)))

246762

## Day 16

### Part 1

In [11]:
with open('inputs/day16.txt') as f:
    s = f.read()[:-1].split('\n')

min_x = 0
min_y = 0
max_x = len(s[0])
max_y = len(s)

def all_matching(grid, char):
    return {(x,y) for y in range(len(grid)) for x in range(len(grid[0])) if grid[y][x] == char}


dash = all_matching(s, '-')
pipe = all_matching(s, '|')
slash = all_matching(s, '/')
backslash = all_matching(s, '\\')


def interact_dash(direction):
    if direction == (0,-1) or direction == (0,1):
        return [(-1,0),(1,0)]
    return [direction]


def interact_pipe(direction):
    if direction == (-1,0) or direction == (1,0):
        return [(0,-1),(0,1)]
    return [direction]


def interact_slash(direction):
    (x,y) = direction
    return [(-y,-x)]


def interact_backslash(direction):
    (x,y) = direction
    return [(y,x)]


def interact(light, direction):
    if light in dash:
        dir = interact_dash(direction)
    elif light in pipe:
        dir = interact_pipe(direction)
    elif light in slash:
        dir = interact_slash(direction)
    elif light in backslash:
        dir = interact_backslash(direction)
    else:
        dir = [direction]
    return [(light,d) for d in dir]


def move_light(light, direction):
    (x,y) = light
    (x_d,y_d) = direction
    if not (x+x_d in range(min_x, max_x) and y+y_d in range(min_y, max_y)):
        return None
    return ((x+x_d, y+y_d),direction)


def tick_single(light,direction):
    new_lights = interact(light,direction)
    return [move_light(light[0],light[1]) for light in new_lights if light is not None]


def tick_all(lights, visited):
    new_lights = set()
    for light,direction in lights:
        for new_light in tick_single(light,direction):
            new_lights.add(new_light)
    new_lights = {light for light in new_lights if light not in visited and light is not None}
    for light in new_lights:
        visited.add((light))
    return (new_lights, visited)


def energy_with_config(position, direction):
    lights = {(position,direction)}
    visited = {(position,direction)}
    
    while len(lights) > 0:
        (lights,visited) = tick_all(lights, visited)

    return len({light for (light,_) in visited})
    

energy_with_config((0,0),(1,0))

8389

### Part 2

In [12]:
north = max(map(lambda x: energy_with_config((x,0),(0,1)), range(min_x,max_x)))
south = max(map(lambda x: energy_with_config((x,max_y-1),(0,-1)), range(min_x,max_x)))
west = max(map(lambda y: energy_with_config((0,y),(1,0)), range(min_y,max_y)))
east = max(map(lambda y: energy_with_config((max_x-1,y),(-1,0)), range(min_y,max_y)))

max([north,south,west,east])

8564

 ## Day 17

### Part 1

In [106]:
with open('inputs/day17.txt') as f:
    s = f.read()[:-1].split('\n')

start = (0,0)
target = (len(s[0])-1, len(s)-1)
min_heat_loss = {(start,0,(0,1)): 0, (start,0,(1,0)): 0}
queue = {(start,0,0,(0,1)),(start,0,0,(1,0))}

def move(current,heat_loss,straight_for,previous_direction):
    if (current,straight_for,previous_direction) in min_heat_loss and \
        min_heat_loss[(current,straight_for,previous_direction)] <= heat_loss and heat_loss != 0:
        return
    (x,y) = current
    min_heat_loss[(current,straight_for,previous_direction)] = heat_loss
    (prev_x,prev_y) = previous_direction
    next_options = [(prev_y,prev_x),(-prev_y,-prev_x)]
    if straight_for < 3:
        next_options.append(previous_direction)
    for dir in next_options:
        (x_d,y_d) = dir
        if x+x_d in range(0,len(s[0])) and y+y_d in range(0,len(s)):
            queue.add(((x+x_d,y+y_d), heat_loss + int(s[y+y_d][x+x_d]), straight_for+1 if dir == previous_direction else 1, dir))

while len(queue) > 0:
    (current, heat_loss, straight_for, previous_direction) = queue.pop()
    move(current, heat_loss, straight_for, previous_direction)

min([min_heat_loss[k] for k in min_heat_loss if k[0] == target])

791

### Part 2

In [107]:
min_heat_loss = {(start,0,(0,1)): 0, (start,0,(1,0)): 0}
queue = {(start,0,0,(0,1)),(start,0,0,(1,0))}

def move_ultra(current,heat_loss,straight_for,previous_direction):
    if (current,straight_for,previous_direction) in min_heat_loss and \
        min_heat_loss[(current,straight_for,previous_direction)] <= heat_loss and heat_loss != 0:
        return
    (x,y) = current
    min_heat_loss[(current,straight_for,previous_direction)] = heat_loss
    (prev_x,prev_y) = previous_direction
    if straight_for < 4:
        next_options = [previous_direction]
    else:
        next_options = []
        if x+4*prev_y in range(0,len(s[0])) and y+4*prev_x in range(0,len(s)):
            next_options.append((prev_y,prev_x))
        if x-4*prev_y in range(0,len(s[0])) and y-4*prev_x in range(0,len(s)):
            next_options.append((-prev_y,-prev_x))
        if straight_for < 10:
            next_options.append(previous_direction)
    for dir in next_options:
        (x_d,y_d) = dir
        if x+x_d in range(0,len(s[0])) and y+y_d in range(0,len(s)):
            queue.add(((x+x_d,y+y_d), heat_loss + int(s[y+y_d][x+x_d]), straight_for+1 if dir == previous_direction else 1, dir))

while len(queue) > 0:
    (current, heat_loss, straight_for, previous_direction) = queue.pop()
    move_ultra(current, heat_loss, straight_for, previous_direction)

min([min_heat_loss[k] for k in min_heat_loss if k[0] == target])

900

## Day 18

### Part 1

In [99]:
with open('inputs/day18.txt') as f:
    s = f.read()[:-1]

instructions = [tuple([line.split(' ')[0], int(line.split(' ')[1])]) for line in s.split('\n')]
directions = {'U': (0,-1), 'D': (0,1), 'L': (-1,0), 'R': (1,0)}


def dig_holes(instructions):
    current = (0,0)
    hole_ranges = set()

    min_x = 0
    max_x = 0
    min_y = 0
    max_y = 0
    
    for (dir,times) in instructions:
        d = directions[dir]
        if dir == 'D' or dir == 'R':
            hole_ranges.add(((current[0],current[0]+times*d[0]), (current[1],current[1]+times*d[1])))
        else:
            hole_ranges.add(((current[0]+times*d[0],current[0]), (current[1]+times*d[1],current[1])))
            
        current = (current[0]+times*d[0], current[1]+times*d[1])
        
        if current[0] < min_x:
            min_x = current[0]
        if current[1] < min_y:
            min_y = current[1]
        if current[0] > max_x:
            max_x = current[0]
        if current[1] > max_y:
            max_y = current[1]
    
    min_x -= 1
    max_x += 1
    min_y -= 1
    max_y += 1

    return hole_ranges


def holes_in_row(hole_ranges,y):
    related_vertical = [r for r in hole_ranges if y in range(r[1][0],r[1][1]+1) if r[0][0] == r[0][1]]
    down = set()
    up = set()
    
    for ((xv,_), (yv1,yv2)) in related_vertical:
        match = False
        if yv1 == y:
            down.add(xv)
            match = True
        if yv2 == y:
            up.add(xv)
            match = True
        if not match:
            down.add(xv)
            up.add(xv)
                
    down = sorted(list(down))
    up = sorted(list(up))
    down_pairs = [(down[i],down[i+1]) for i in range(0,len(down),2)]
    up_pairs = [(up[i],up[i+1]) for i in range(0,len(up),2)]
    down_sum = sum([h2-h1+1 for (h1,h2) in down_pairs])
    up_sum = sum([h2-h1+1 for (h1,h2) in up_pairs])
    overlaps = []
    for (d1,d2) in down_pairs:
        for (u1,u2) in up_pairs:
            if d1 in range(u1,u2+1) and d2 in range(u1,u2+1):
                overlaps.append((d1,d2))
            elif u1 in range(d1,d2+1) and u2 in range(d1,d2+1):
                overlaps.append((u1,u2))
            elif d1 in range(u1,u2+1):
                overlaps.append((d1,u2))
            elif u1 in range(d1,d2+1):
                overlaps.append((u1,d2))
    overlap_sum = sum([h2-h1+1 for (h1,h2) in overlaps])
    return down_sum + up_sum - overlap_sum


def count_holes(hole_ranges):  
    horizontal = sorted(list({r[1][0] for r in hole_ranges if r[1][0] == r[1][1]}))
    acc = 0
    for i,y in enumerate(horizontal):
        acc += holes_in_row(hole_ranges,y)
        if i < len(horizontal)-1:
            acc += holes_in_row(hole_ranges,y+1)*(horizontal[i+1]-y-1)
    return acc

hole_ranges = dig_holes(instructions)
count_holes(hole_ranges)

108909

### Part 2

In [100]:
hex_directions = {'0': 'R', '1': 'D', '2': 'L', '3': 'U'}
hex_instructions = [tuple([hex_directions[line.split(' ')[2][-2]], int(line.split(' ')[2][2:-2], 16)]) for line in s.split('\n')]

hole_ranges = dig_holes(hex_instructions)
count_holes(hole_ranges)

133125706867777

## Day 19

### Part 1

In [113]:
from functools import reduce

with open('inputs/day19.txt') as f:
    s = f.read()[:-1].split('\n\n')


def get_threshold(rule):
    return int(rule[2:].split(':')[0])


def get_destination(rule):
    destination = rule.split(':')[-1]
    if destination == 'A':
        return True
    if destination == 'R':
        return False
    return destination


def add_rule(rules,new_rule):
    if ':' not in new_rule:
        rules.append((None,get_destination(new_rule)))
        return rules
        
    threshold = get_threshold(new_rule)
    if '<' in new_rule:
        condition_sym = '<'
        condition_f = lambda n: n < threshold
    elif '>' in new_rule:
        condition_sym = '>'
        condition_f = lambda n: n > threshold
        
    condition = (new_rule[0], condition_f, threshold, condition_sym)
    destination = get_destination(new_rule)
    rule = (condition, destination)
    rules.append(rule)
    return rules


def parse_workflow(workflow):
    return reduce(lambda rs,r: add_rule(rs,r), workflow, [])
    

def run_workflow(part, workflow):
    for (condition, destination) in workflow:
        if condition is not None and condition[1](part[condition[0]]):
            return destination
        elif condition is None:
            return destination


def check_part(part, workflows):
    current = 'in'
    while current != True and current != False:
        current = run_workflow(part, workflows[current])
    return current
        

workflows = {workflow.split('{')[0]: parse_workflow(workflow[:-1].split('{')[1].split(',')) for workflow in s[0].split('\n')}
parts = [{rating.split('=')[0]: int(rating.split('=')[1]) for rating in part[1:-1].split(',')} for part in s[1].split('\n')]
        
selected_parts = [part for part in parts if check_part(part,workflows)]
sum([sum(part.values()) for part in selected_parts])

263678

### Part 2

In [114]:
def split_rule_sets_on_rule(rule, rule_sets):
    (category, _, split_on, sym) = rule
    matching = []
    non_matching = []
    for rule_set in rule_sets:
        if sym == '<':
            if rule_set[category][-1] < split_on:
                matching.append(rule_set)
            elif rule_set[category][0] >= split_on:
                non_matching.append(rule_set)
            else:
                matching.append({cat: rule_set[cat] if cat != category else (rule_set[cat][0], split_on-1) for cat in rule_set})
                non_matching.append({cat: rule_set[cat] if cat != category else (split_on, rule_set[cat][-1]) for cat in rule_set})
        elif sym == '>':
            if rule_set[category][0] > split_on:
                matching.append(rule_set)
            elif rule_set[category][-1] <= split_on:
                non_matching.append(rule_set)
            else:
                matching.append({cat: rule_set[cat] if cat != category else (split_on+1, rule_set[cat][-1]) for cat in rule_set})
                non_matching.append({cat: rule_set[cat] if cat != category else (rule_set[cat][0], split_on) for cat in rule_set})
    return (matching, non_matching)
            
            
def create_rule_sets_from(rule_sets, workflow_name):
    workflow = workflows[workflow_name]
    matching = []
    non_matching = rule_sets
    for (condition, destination) in workflow:
        if condition is not None:
            (match, non_matching) = split_rule_sets_on_rule(condition, non_matching)
        else:
            match = non_matching
            
        if destination not in {True, False}:
            match = create_rule_sets_from(match, destination)
        elif not destination:
            match = []
            
        matching.extend(match)

    return matching


def rule_set_value(rule_set):
    return reduce(lambda acc,x: acc*x, [end-start+1 for (start, end) in rule_set.values()], 1)


rule_sets = create_rule_sets_from([{'x': (1,4000), 'm': (1,4000), 'a': (1,4000), 's': (1,4000)}], 'in')
sum([rule_set_value(rule_set) for rule_set in rule_sets])

125455345557345

## Day 20

### Day 1

In [390]:
with open('inputs/day20.txt') as f:
    s = f.read()[:-1].split('\n')


class Module:
    def __init__(self,name,inputs,outputs):
        self.name = name
        self.state = None
        self.inputs = {i:None for i in inputs}
        self.outputs = {o:None for o in outputs}
        self.type = ''
        self.low = 0
        self.high = 0

    def send(self,pulse):
        if pulse is not None:
            for module in self.outputs:
                self.outputs[module] = pulse
                if pulse == True:
                    self.high += 1
                else:
                    self.low += 1

    def receive(self,pulse,source):
        self.inputs[source] = pulse

    def process_pulses(self):
        pass

    def process(self):
        self.outputs = {o:None for o in self.outputs}
        self.process_pulses()
        self.inputs = {i:None for i in self.inputs}

    def __repr__(self):
        return  ','.join(self.inputs) + " -> " + self.type + self.name + " -> " + ','.join(self.outputs)


class FlipFlop(Module):
    def __init__(self,name,inputs,outputs):
        super().__init__(name,inputs,outputs)
        self.state = False
        self.type = '%'
    
    def process_pulses(self):
        if False in self.inputs.values():
            self.state = not self.state
            self.send(self.state)


class Conjunction(Module):
    def __init__(self,name,inputs,outputs):
        super().__init__(name,inputs,outputs)
        self.state = {i:False for i in inputs}
        self.type = '&'

    def process_pulses(self):
        count = 0
        for i in self.inputs:
            if self.inputs[i] is not None:
                self.state[i] = self.inputs[i]
                count += 1

        for _ in range(count):
            if False in self.state.values():
                self.send(True)
            else:
                self.send(False)


class Broadcast(Module):
    def __init__(self,outputs):
        super().__init__("broadcaster", ["button"], outputs)

    def process_pulses(self):
        self.send(list(self.inputs.values())[0])


class Button(Module):
    def __init__(self):
        super().__init__("button", [], ["broadcaster"])
        self.state = False

    def process_pulses(self):
        if self.state:
            self.send(False)
        self.state = False

    def press(self):
        self.state = True


input_map = {}
for line in s:
    outputs = line.split(' -> ')[-1].split(', ')
    for o in outputs:
        if o not in input_map:
            input_map[o] = []
        if line[0] in {'%', '&'}:
            input_map[o].append(line.split(' -> ')[0][1:])
        else:
            input_map[o].append(line.split(' -> ')[0])

def init_modules():
    modules = {"button": Button()}
    for line in s:
        module, outputs = line.split(' -> ')
        outputs = outputs.split(', ')
        if module == 'broadcaster':
            modules[module] = Broadcast(outputs)
        else:
            module = module[1:]
        if line[0] == '%':
            modules[module] = FlipFlop(module, input_map[module], outputs)
        elif line[0] == '&':
            modules[module] = Conjunction(module, input_map[module], outputs)
    return modules


def tick(modules):
    changed = False
    for name in modules:
        for o in modules[name].outputs:
            pulse = modules[name].outputs[o]
            if pulse is not None:
                changed = True 
                if o in modules:
                    modules[o].receive(pulse, name)

    for module in modules.values():
        module.process()
        
    return changed


def push_button(modules):
    modules["button"].press()
    tick(modules)
    changed = True
    while changed:
        changed = tick(modules)

def count_pulses(modules):
    low = 0
    high = 0
    for module in modules.values():
        low += module.low
        high += module.high
    return (low,high)

modules = init_modules()

for _ in range(1000):
    push_button(modules)

low, high = count_pulses(modules)

low*high

819397964

### Day 2

In [391]:
from functools import reduce
from math import lcm

def identify_core(circuit):
    flip_flops = {module for module in circuit if module in modules and modules[module].type == '%'}
    center = [module for module in circuit if module in modules and modules[module].type == '&' and list(modules[module].inputs)[0] in flip_flops][0]
    return (flip_flops, center)
    

def identify_circuit(start, modules):
    queue = {start}
    circuit = set()
    while len(queue) > 0:
        current = queue.pop()
        circuit.add(current)
        if current in modules:
            for o in modules[current].outputs:
                if o not in circuit:
                    queue.add(o)
    return identify_core(circuit)  


def circuit_state(circuit,modules):
    (flip_flops, _) = circuit
    return tuple([modules[ff].state for ff in sorted(list(flip_flops))])


def find_loop(circuit):
    modules = init_modules()
    states = []
    current = circuit_state(circuit, modules)
    while current not in states:
        states.append(current)
        push_button(modules)
        current = circuit_state(circuit, modules)
    return len(states) - states.index(current)


circuits = [identify_circuit(module, modules) for module in modules['broadcaster'].outputs]
loop_lengths = [find_loop(circuit) for circuit in circuits]

reduce(lambda acc,n: lcm(acc,n), loop_lengths, 1)

252667369442479

## Day 21

### Part 1

In [392]:
with open('inputs/day21.txt') as f:
    s = f.read()[:-1].split('\n')

start = [(x,y) for x in range(len(s[0])) for y in range(len(s)) if s[y][x] == 'S'][0]
plots = {(x-start[0],y-start[1]) for x in range(len(s[0])) for y in range(len(s)) if s[y][x] != '#'}

def neighbours(x,y):
    return {(x+xd, y+yd) for (xd,yd) in {(-1,0),(1,0),(0,-1),(0,1)} if ((x+xd), (y+yd)) in plots}


def next_plots(current,next):
    new_plots = set()
    for (x,y) in current:
        for c in neighbours(x,y):
            if c not in next:
                new_plots.add(c)
    return new_plots,current


def reachable(steps,start=(0,0)):
    even_ring = {start}
    odd_ring = set()
    even = 1
    odd = 0
    
    for i in range(1,steps+1):
        if i % 2 == 0:
            even_ring,odd_ring = next_plots(odd_ring,even_ring)
            even += len(even_ring)
        else:
            odd_ring,even_ring = next_plots(even_ring,odd_ring)
            odd += len(odd_ring)

    return (even,odd)
    

reachable(64)[0]

3532

### Part 2

In [393]:
target = 26501365
length = len(s)
half = length//2
reached_gardens_1d = target//length
inner_gardens = reached_gardens_1d**2+(reached_gardens_1d-1)**2
inner_normal = (int(reached_gardens_1d/2)*2)**2 + (reached_gardens_1d % 2)*4*(reached_gardens_1d-1)
inner_swapped = inner_gardens - inner_normal
corner_index = reached_gardens_1d % 2
edge_index = (reached_gardens_1d+1) % 2

(fully_explored_even, fully_explored_odd) = reachable(131)

inner_normal_total = inner_normal*fully_explored_even
inner_swapped_total = inner_swapped*fully_explored_odd
corners = sum([reachable(131,start)[corner_index] for start in [(half,0), (-half,0), (0,half), (0,-half)]])
edges_small = reached_gardens_1d*sum([reachable(65,start)[corner_index] for start in [(half,half), (half,-half), (-half,half), (-half,-half)]])
edges_large = (reached_gardens_1d-1)*sum([reachable(131+65,start)[edge_index] for start in [(half,half), (half,-half), (-half,half), (-half,-half)]])

inner_normal_total+inner_swapped_total+corners+edges_small+edges_large

590104708070703

## Day 22

### Part 1

In [62]:
with open('inputs/day22.txt') as f:
    s = f.read()[:-1]

bricks = [tuple([tuple([int(n) for n in end.split(',')]) for end in line.split('~')]) for line in s.split('\n')]

def brick_parts(brick):
    end1,end2 = brick
    x1,y1,z1 = end1
    x2,y2,z2 = end2
    return [(x,y,z) for x in range(x1,x2+1) for y in range(y1,y2+1) for z in range(z1,z2+1)]


def final_position(brick, highest_point):
    highest = 0
    for (x,y,_) in brick_parts(brick):
        below = highest_point.get((x,y),0)
        if below > highest:
            highest = below
    
    fall_for = brick[0][2] - (highest+1)
    final_brick = ((brick[0][0],brick[0][1],brick[0][2]-fall_for),(brick[1][0],brick[1][1],brick[1][2]-fall_for))
    final_parts = brick_parts(final_brick)
    
    for (x,y,_) in final_parts:
        highest_point[(x,y)] = brick[1][2] - fall_for
    
    return final_brick


def drop_bricks(bricks):
    bricks = sorted(bricks, key=lambda brick: brick[0][2])
    highest_point = {}
    bricks_final = [final_position(brick, highest_point) for brick in bricks]

    return bricks_final


bricks_final = sorted(drop_bricks(bricks), key=lambda brick: brick[0][2])
count = 0
for brick in bricks_final:
    missing = [b for b in bricks_final if b != brick]
    if missing == drop_bricks(missing):
        count += 1

count

393

### Part 2

In [63]:
fall_sum = 0
for brick in bricks_final:
    missing = [b for b in bricks_final if b != brick]
    missing_final = drop_bricks(missing)
    for i in range(len(missing)):
        if missing[i] != missing_final[i]:
            fall_sum += 1

fall_sum

58440

## Day 23

### Part 1

In [49]:
with open('inputs/day23.txt') as f:
    s = f.read()[:-1].split('\n')

path = {(x,y): s[y][x] for y in range(len(s)) for x in range(len(s[0])) if s[y][x] != '#'}
start = (s[0].index('.'), 0)
finish = (s[-1].index('.'), len(s)-1)


def is_valid_move(current, candidate, visited, path):
    xd = candidate[0]-current[0]
    yd = candidate[1]-current[1]
    
    if candidate not in path:
        return False
    if candidate in visited:
        return False
    if (xd,yd) == (-1,0) and path[candidate] == '>':
        return False
    if (xd,yd) == (1,0) and path[candidate] == '<':
        return False
    if (xd,yd) == (0,-1) and path[candidate] == 'v':
        return False
    if (xd,yd) == (0,1) and path[candidate] == '^':
        return False
    else:
        return True


def next_steps(current, visited, path):
    (x,y) = current
    if path[current] == '>':
        return [(x+1,y)]
    elif path[current] == '<':
        return [(x-1,y)]
    elif path[current] == 'v':
        return [(x,y+1)]
    elif path[current] == '^':
        return [(x,y-1)]
    else:
        return [(x+xd,y+yd) for (xd,yd) in [(-1,0),(1,0),(0,-1),(0,1)] if is_valid_move(current, (x+xd,y+yd), visited, path)]


def move(current, visited, path, longest, seen, queue):
    if (current, visited) in seen:
        return

    seen.add((current,visited))
    if current not in longest or longest[current] < len(visited):
        longest[current] = len(visited)
    
    visited_n = [v for v in visited]
    visited_n.append(current)
    visited_n.sort()
    visited_n = tuple(visited_n)
    for n in next_steps(current, visited_n, path):
        queue.add((n, visited_n))


def find_longest(start, finish, path):
    longest = {}
    seen = set()
    queue = set()
    queue.add((start, ()))
    while len(queue) > 0:
        (current, visited) = queue.pop()
        move(current, visited, path, longest, seen, queue)
    
    return longest[finish]


find_longest(start, finish, path)

2050

### Part 2

In [53]:
with open('inputs/day23.txt') as f:
    s = f.read()[:-1].split('\n')


def is_crossroads(coord, path):
    (x,y) = coord
    if (x,y) not in path:
        return False
    return ((x-1,y) in path and (x+1,y) in path and (x,y-1) in path) or \
            ((x-1,y) in path and (x+1,y) in path and (x,y+1) in path) or \
            ((x,y-1) in path and (x,y+1) in path and (x-1,y) in path) or \
            ((x,y-1) in path and (x,y+1) in path and (x+1,y) in path)


def is_valid_move(current, candidate, previous, path):
    return candidate in path and candidate != previous


def next_steps(current, previous, path):
    (x,y) = current
    return [(x+xd,y+yd) for (xd,yd) in [(-1,0),(1,0),(0,-1),(0,1)] if is_valid_move(current, (x+xd,y+yd), previous, path)]


def walk_path(current, previous, poi, path):
    length = 1
    next_options = next_steps(current, previous, path)
    while len(next_options) == 1:
        previous = current
        current = next_options[0]
        next_options = next_steps(current, previous, path)
        length += 1
    if current in poi:
        return (current, length)


def distances(current, poi, path):
    directions = next_steps(current, None, path)
    neighbours = {}
    for d in directions:
        neighbour = walk_path(d, current, poi, path)
        if neighbour is not None:
            neighbours[neighbour[0]] = neighbour[1]
    return neighbours


def walk_full(current, dist, visited, target, length):
    if current == target:
        return length

    options = [opt for opt in dist[current] if opt not in visited]
    if len(options) == 0:
        return 0
        
    max_subpath = max([walk_full(opt, dist, visited + [current], target, length + dist[current][opt]) for opt in options])

    return max_subpath


path_no_slope = {p: '.' for p in path}
poi = {c for c in path_no_slope if is_crossroads(c, path) or c == start or c == finish}

dist = {}
for p in list(poi):
    dist[p] = distances(p, poi, path_no_slope)

walk_full(start, dist, [], finish, 0)

6262

## Day 24

### Part 1

In [1]:
with open('inputs/day24.txt') as f:
    s = f.read()[:-1].split('\n')


def parse_coords(coords):
    return tuple([int(c) for c in coords.split(', ')])


def parse_hail(line):
    coords = line.split(' @ ')
    return parse_coords(coords[0]), parse_coords(coords[1])


def collides_at_2d(hail1, hail2):
    (h01, ht1) = hail1
    (h02, ht2) = hail2
    a1 = ht1[1]
    a2 = ht2[1]
    b1 = -ht1[0]
    b2 = -ht2[0]
    c1 = ht1[0]*h01[1] - ht1[1]*h01[0]
    c2 = ht2[0]*h02[1] - ht2[1]*h02[0]
    if (a1*b2-a2*b1) == 0:
        return None
    x = (b1*c2-b2*c1)/(a1*b2-a2*b1)
    y = (a2*c1-a1*c2)/(a1*b2-a2*b1)

    if (x-h01[0])/ht1[0] < 0 or (y-h01[1])/ht1[1] < 0 or (x-h02[0])/ht2[0] < 0 or (y-h02[1])/ht2[1] < 0:
        return None
    
    return (x,y)


hail = [parse_hail(line) for line in s]
area_start = 200000000000000
area_end = 400000000000000

count = 0
for i in range(len(hail)):
    for j in range(i+1, len(hail)):
        collision = collides_at_2d(hail[i], hail[j])
        if collision is not None:
            (x,y) = collision
            if x >= area_start and x <= area_end and  y >= area_start and y <= area_end:
                count += 1

count

13754

### Part 2

In [3]:
import numpy as np

# There are rounding errors for the first 3
(p1,v1) = hail[1]
(p2,v2) = hail[2]
(p3,v3) = hail[3]

A = np.array([
    [v2[1]-v1[1], v1[0]-v2[0], 0,           p1[1]-p2[1], p2[0]-p1[0], 0],
    [v3[1]-v1[1], v1[0]-v3[0], 0,           p1[1]-p3[1], p3[0]-p1[0], 0],
    [0,           v2[2]-v1[2], v1[1]-v2[1], 0,           p1[2]-p2[2], p2[1]-p1[1]],
    [0,           v3[2]-v1[2], v1[1]-v3[1], 0,           p1[2]-p3[2], p3[1]-p1[1]],
    [v2[2]-v1[2], 0,           v1[0]-v2[0], p1[2]-p2[2], 0,           p2[0]-p1[0]],
    [v3[2]-v1[2], 0,           v1[0]-v3[0], p1[2]-p3[2], 0,           p3[0]-p1[0]]])

b = [(p1[1]*v1[0] - p2[1]*v2[0]) - (p1[0]*v1[1] - p2[0]*v2[1]),
     (p1[1]*v1[0] - p3[1]*v3[0]) - (p1[0]*v1[1] - p3[0]*v3[1]),
     (p1[2]*v1[1] - p2[2]*v2[1]) - (p1[1]*v1[2] - p2[1]*v2[2]),
     (p1[2]*v1[1] - p3[2]*v3[1]) - (p1[1]*v1[2] - p3[1]*v3[2]),
     (p1[2]*v1[0] - p2[2]*v2[0]) - (p1[0]*v1[2] - p2[0]*v2[2]),
     (p1[2]*v1[0] - p3[2]*v3[0]) - (p1[0]*v1[2] - p3[0]*v3[2])]

x = np.linalg.solve(A, b)
int(sum(x[:3]))

711031616315001

## Day 25

In [36]:
import random
from functools import reduce

with open('inputs/day25.txt') as f:
    s = f.read()[:-1].split('\n')


connections = {}
for line in s:
    parts = line.split(': ')
    if parts[0] not in connections:
        connections[parts[0]] = set()
    for p in parts[1].split(' '):
        if p not in connections:
            connections[p] = set()
        connections[parts[0]].add(p)
        connections[p].add(parts[0])


def connected_to(start, connections):
    queue = {start}
    connected = {}
    while len(queue) > 0:
        current = queue.pop()
        connected[current] = connected.get(current, 0) + 1
        for nxt in connections[current]:
            if nxt not in connected:
                queue.add(nxt)
    return connected


def merge(node1, node2, connections):
    if node1 not in connections[node2] or node2 not in connections[node1]:
        return connections
    connections[node1] = [node for node in connections[node1] if node != node2]
    connections[node2] = [node for node in connections[node2] if node != node1]
    node_conn = connections[node1] + connections[node2]
    del connections[node1]
    del connections[node2]
    new_node = node1 + '-' + node2
    connections[new_node] = node_conn
    for node in connections:
        connections[node] = [n if n != node1 and n != node2 else new_node for n in connections[node]]
    return connections


def pick_nodes(connections):
    node1 = random.choice(list(connections.keys()))
    node2 = random.choice(list(connections[node1]))
    return node1, node2


def find_cut(connections):
    conn = connections
    while len(conn) > 2:
        node1, node2 = pick_nodes(conn)
        conn = merge(node1, node2, conn)
    return conn


def find_min_cut(connections):
    conn = connections.copy()
    conn = find_cut(conn)
    while len(list(conn.values())[0]) > 3:
        conn = connections.copy()
        conn = find_cut(conn)
    return conn


conn = find_min_cut(connections)
reduce(lambda acc, x: acc*x, map(lambda x: x.count('-')+1, conn.keys()), 1)

598120